# Classification binaire de vins blancs

On cherche à prédire la note de vins rouges.
On utilisera l'algorithme kNN (classifieur) avec une recherche sur grille ainsi qu'une fonction de validation croisée implémentées manuellement puis comparées avec celles utilisées par scikit-learn.
La comparaison se fait sur la précision des modèles entraînés.

In [1]:
import numpy as np
import pandas as pd

data = pd.read_csv('winequality-white.csv', sep=";")

In [2]:
# Visualisation des données
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [3]:
# Extraction des données
X = data.as_matrix(data.columns[:-1])
y = data.as_matrix([data.columns[-1]])
y = y.flatten()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [4]:
X.shape

(4898, 11)

In [5]:
y.shape

(4898,)

In [6]:
# Analyse des notes possibles pour les vins
np.unique(y)

array([3, 4, 5, 6, 7, 8, 9], dtype=int64)

### Fonction de validation croisée "maison"

In [7]:
# Séparation des données en jeux d'entraînement et de test
from sklearn import model_selection
X_train_total, X_test, y_train_total, y_test = \
model_selection.train_test_split(X, y,
                                test_size=0.25 # 25% des données dans le jeu de test
                                )

In [8]:
# Standardisation des données X
from sklearn import preprocessing
std_scale = preprocessing.StandardScaler().fit(X_train_total)
X_train_total = std_scale.transform(X_train_total)
X_test = std_scale.transform(X_test)

In [9]:
from kNN_classifier_fonction import Validation_croisee

# Paramètres à choisir, ici le nombre de voisins :
n_neighbors = [3, 5, 7, 9, 11, 13, 15]

# Nombre de folds à choisir : 
cv = 5

# Retour de la position du meilleur K dans la liste n_neighbors grâce à la fonction :
K_opt = Validation_croisee(X_train_total, y_train_total, n_neighbors, cv)

Nombre de voisins : 3
Numéro de fold de test : 1
Accurracy du fold : 53.06 %
Numéro de fold de test : 2
Accurracy du fold : 55.78 %
Numéro de fold de test : 3
Accurracy du fold : 54.83 %
Numéro de fold de test : 4
Accurracy du fold : 51.56 %
Numéro de fold de test : 5
Accurracy du fold : 54.16 %
Accurracy moyenne du modèle '3 voisins' : 53.88 %
Nombre de voisins : 5
Numéro de fold de test : 1
Accurracy du fold : 52.79 %
Numéro de fold de test : 2
Accurracy du fold : 54.01 %
Numéro de fold de test : 3
Accurracy du fold : 55.10 %
Numéro de fold de test : 4
Accurracy du fold : 53.06 %
Numéro de fold de test : 5
Accurracy du fold : 53.89 %
Accurracy moyenne du modèle '5 voisins' : 53.77 %
Nombre de voisins : 7
Numéro de fold de test : 1
Accurracy du fold : 53.88 %
Numéro de fold de test : 2
Accurracy du fold : 52.65 %
Numéro de fold de test : 3
Accurracy du fold : 54.29 %
Numéro de fold de test : 4
Accurracy du fold : 53.06 %
Numéro de fold de test : 5
Accurracy du fold : 51.84 %
Accurracy

In [10]:
# Test du modèle le plus performant
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors = K_opt)
model.fit(X_train_total, y_train_total)  #fit the model

y_pred = model.predict(X_test)
result = accuracy_score(y_test, y_pred) * 100
print("Meilleur hyperparamètre : {}".format(K_opt))
print("\nSur le jeu de test : {:0.2f} %".format(result))

Meilleur hyperparamètre : 13

Sur le jeu de test : 54.61 %


### Fonction GridsearchCV implémentée dans Scikit

In [11]:
from sklearn import model_selection
from sklearn import neighbors, metrics

# Fixer les valeurs des hyperparamètres à tester
param_grid = {'n_neighbors':[3, 5, 7, 9, 11, 13, 15]}

# Choisir un score à optimiser, ici l'accuracy (proportion de prédictions correctes)
score = 'accuracy'

# Non randomisation des folds (shuffle est normalement "False" par défaut)
cv_GridSearch=model_selection.KFold(n_splits=5,shuffle=False)

# Créer un classifieur kNN avec recherche d'hyperparamètre par validation croisée
clf = model_selection.GridSearchCV(neighbors.KNeighborsClassifier(), # un classifieur kNN
param_grid, # hyperparamètres à tester
cv=cv_GridSearch, # nombre de folds de validation croisée
scoring=score # score à optimiser
)

# Optimiser ce classifieur sur le jeu d'entraînement
clf.fit(X_train_total, y_train_total)

# Afficher le(s) hyperparamètre(s) optimaux
print("Meilleur(s) hyperparamètre(s) sur le jeu d'entraînement:"),
print(clf.best_params_)

# Afficher les performances correspondantes
print("Résultats de la validation croisée :")
for mean, std, params in zip(clf.cv_results_['mean_test_score'], # score moyen
clf.cv_results_['std_test_score'], # écart-type du score
clf.cv_results_['params'] # valeur de l'hyperparamètre
):
    print("\t%s = %0.3f (+/-%0.03f) for %r" % (score, # critère utilisé
    mean, # score moyen
    std * 2, # barre d'erreur
    params # hyperparamètre
    ))

Meilleur(s) hyperparamètre(s) sur le jeu d'entraînement:
{'n_neighbors': 13}
Résultats de la validation croisée :
	accuracy = 0.539 (+/-0.030) for {'n_neighbors': 3}
	accuracy = 0.538 (+/-0.017) for {'n_neighbors': 5}
	accuracy = 0.531 (+/-0.017) for {'n_neighbors': 7}
	accuracy = 0.534 (+/-0.015) for {'n_neighbors': 9}
	accuracy = 0.538 (+/-0.034) for {'n_neighbors': 11}
	accuracy = 0.541 (+/-0.039) for {'n_neighbors': 13}
	accuracy = 0.534 (+/-0.025) for {'n_neighbors': 15}


In [12]:
# Test du modèle le plus performant
y_pred = clf.predict(X_test)
print("Meilleur hyperparamètre : {}".format(clf.best_params_))
result = accuracy_score(y_test, y_pred) * 100
print("Sur le jeu de test : {:0.2f} %".format(result))

Meilleur hyperparamètre : {'n_neighbors': 13}
Sur le jeu de test : 54.61 %
